## **Data Collection**

In [12]:
import requests
import pandas as pd
import json
from lxml import etree
import numpy as np
import zipfile
import os
from bs4 import BeautifulSoup


In [13]:
# create a folder to serve for store data
folder = "data"
os.makedirs(folder, exist_ok=True)

### **1. Crawl from website Tatoeba**

Source: https://tatoeba.org/

In [ ]:

def crawl_data(leng_min=1, page_max=100):
    engs, vies = [], []

    for page in range(1, page_max + 1):
        try:
            url = "https://tatoeba.org/vi/api_v0/search?from=eng&to=vie&trans_filter=limit&trans_to=vie&page={}&word_count_max=&word_count_min={}".format(page, leng_min)
            response = requests.get(url)

            if response.status_code == 200:
              data = response.json()
              for item in data.get('results', []):
                  english = item['text']
                  engs.append(english)

                  translations = item.get('translations', [])
                  vietnamese_translations = []

                  for sublist in translations:
                      for trans in sublist:
                          if trans['lang'] == 'vie':
                              vietnamese_translations.append(trans['text'])

                  if vietnamese_translations:
                      longest_vi = min(vietnamese_translations, key=len)
                      vies.append(longest_vi)

        except Exception as e:
            print(f"Error on page {page}: {e}")

    return engs, vies

In [ ]:
def crawl_all(leng_max = 10):
  engs_all, vies_all = [],[]
  for e in range(1, leng_max + 1):
    try:
      engs, vies = crawl_data(leng_min = e, page_max = 100)
      engs_all.extend(engs)
      vies_all.extend(vies)
    except Exception as ex:
      print(f"Error on at least {e} words: {ex}")
  df = pd.DataFrame({'English': engs_all, 'Vietnamese': vies_all})
  return df

df1 = crawl_all(20)
df1.head(10)

,English,Vietnamese
0,Hmm?,Hử?
1,Translate.,Dịch.
2,Faster.,Nhanh hơn.
3,Kiss!,Hôn đi!
4,Please...,Đi mà...
...,...,...
15856,My friend from university gave me a Kindle bec...,Một người bạn thời đại học đã cho tôi một cái ...
15857,In all the work I have done as President—every...,Trong tất cả những công việc mà tôi đã làm với...
15858,The 26th of September is the European Day of L...,Ngày 26 tháng chín là ngày Ngôn ngữ của Châu Â...
15859,While the Kabyle people were fighting French c...,Trong lúc những người Kabyle đang chiến đấu vớ...


In [ ]:
df1.to_csv(os.path.join(folder,"tatoeba.csv"), index= False)

In [ ]:
df1.shape

(15861, 2)

### **2. Download from Opus**

please visit [Opus](https://opus.nlpl.eu/) to understand that in details


Link download the file: https://object.pouta.csc.fi/OPUS-QED/v2.0a/moses/en-vi.txt.zip

In [ ]:
des_file = os.path.join(folder,"en_vi")

In [19]:
with zipfile.ZipFile(os.path.join(folder,"en-vi.txt.zip"), "r") as zip_ref:
    zip_ref.extractall(des_file) 
    print("Files:", zip_ref.namelist())

Files: ['README', 'LICENSE', 'QED.en-vi.en', 'QED.en-vi.vi', 'QED.en-vi.xml']


In [20]:
engs, vies = [],[]
with open(os.path.join(des_file,"QED.en-vi.en"), "r", encoding="utf-8") as f_en, \
    open(os.path.join(des_file,"QED.en-vi.vi"), "r", encoding="utf-8") as f_vi:
    
    for i, (en_line, vi_line) in enumerate(zip(f_en, f_vi)):
        en = en_line.strip()
        engs.append(en)
        
        vi = vi_line.strip()
        vies.append(vi)

In [21]:
for i in range(10):
    print(f"eng sentence: {engs[i]}\nviet sentence: {vies[i]}\n") 

eng sentence: It should print 12, and 3 plus 4 plus 5 with all the spaces.
viet sentence: Nó sẽ in ra 12, và 3 + 4 + 5 với tất cả các khoảng trắng.

eng sentence: This might have been different from what you expected.
viet sentence: Điều này có lẽ khác với những gì bạn mong đợi.

eng sentence: We didn't use quotation marks here.
viet sentence: Chúng ta không sử dụng dấu ngoặc kép ở đây.

eng sentence: The quotation marks mark literal text.
viet sentence: Dấu ngoặc kép dùng để đánh dấu văn bản.

eng sentence: If we use quotes, it means that we want to write 3 plus 4 plus 5 as it is.
viet sentence: Nếu ta sử dụng ngoặc kép, điều đó có nghĩa là chúng ta muốn viết ra nguyên văn 3 + 4 + 5.

eng sentence: But when we don;t use quotes, the computer reads 3 plus 4 plus 5 as numbers.
viet sentence: Ngược lại, nếu chúng ta không dùng dấu ngoặc, máy tính xem 3 + 4 + 5 như những con số.

eng sentence: It knows how to add the numbers, so it added the numbers and then it printed the result as you ca

In [22]:
len(engs)

338024

In [ ]:
df2 = pd.DataFrame(
    {'English': engs, 'Vietnamese': vies}
)

In [24]:
df2.head(5)

,English,Vietnamese
0,"It should print 12, and 3 plus 4 plus 5 with a...","Nó sẽ in ra 12, và 3 + 4 + 5 với tất cả các kh..."
1,This might have been different from what you e...,Điều này có lẽ khác với những gì bạn mong đợi.
2,We didn't use quotation marks here.,Chúng ta không sử dụng dấu ngoặc kép ở đây.
3,The quotation marks mark literal text.,Dấu ngoặc kép dùng để đánh dấu văn bản.
4,"If we use quotes, it means that we want to wri...","Nếu ta sử dụng ngoặc kép, điều đó có nghĩa là ..."


In [26]:
df2.to_csv(os.path.join(folder,"QED.csv"), index=False)